In [ ]:
import sys
sys.path.append('/data/ECS/PythonTutorial/')
import log
log.topic_id = '19_VeritabaniIslemleri-cx_Oracle'

# Veritabanı İşlemleri (cx_Oracle)

In [ ]:
import cx_Oracle

## şifre dosyası oluşturma

In [ ]:
import os

kullanici_adi = 'alperayd'
sifre = 'xxxxx'
path = f'/data/home/{kullanici_adi}/oracle.txt'

with open(path,'w') as f:
    f.write(f'{kullanici_adi};{sifre}')
    
os.chmod(path, 0o600)

## connect

In [ ]:
import cx_Oracle
import os

def connect_db(user_name=None, password=None):
    # -----------------------------------------
    kullanici_adi = 'alperayd'
    # -----------------------------------------
    
    try:
        with open(f'/data/home/{kullanici_adi}/oracle.txt','r') as f:
            r = f.read()
            user_name = r.split(';')[0]
            password = r.split(';')[1]
        
        os.environ['ORACLE_HOME'] = '/data/app/oracle/product/12.2.0/client_1'
        host = "pgarsnb1rw.fw.garanti.com.tr:4520/pgarsnb1b"
        #host_dwh  = 'pgardwhrw:4520/pgardwh'
        #host_qgar = 'qgardwhapp.fw.garanti.com.tr:4520/QGARDWH'
        
        conn = cx_Oracle.connect(user_name, password, host, encoding='UTF-8')
        return conn
    except Exception as ex:
        print(f"bağlantı sağlanamadı, \nHata: {ex}")
        return None

conn =connect_db()
print(conn)

### tabloya yetki verme

In [ ]:
def tabloya_yetki_ver(tbl_name, role):
    conn = connect_db()
    curr = conn.cursor()

    sql = f"""begin DS_GRANT_TO_OTHERGROUP ('{tbl_name}','{role}'); end;""" 
    curr.execute(sql)
    conn.close()
    
    
    
    
for role in ['BRYSLRSKSTRTJLR5579','TCRRSKSTRTJLR3966','MSTRCZMLRVRVNLTK99992361']:
    tabloya_yetki_ver('KLV_MUSTERI_GNL', role)


### açık kalan session'ların kapatılması

select * from PGARDBA.DS_MY_SESSIONS order by MY_SQL_START_TIME desc;

## veri okuma

### fetch komutu ile okuma

In [ ]:
# veritabanına bağlan
conn = connect_db()

# sql ile verileri çekme
sql = """
    select * 
    from pgaredm.v_cnf_customer_hst 
    where record_date=trunc(sysdate-1) 
        and rownum<1000
"""
curr = conn.cursor()
curr.execute(sql)

ret = curr.fetchmany(numRows=50)
col_names = [row[0] for row in curr.description]
    
print(curr.description)
print(col_names)

curr.close()
conn.close()

In [ ]:
# veritabanına bağlan
conn = connect_db()

# sql ile verileri çekme
sql = """
    select customer_tp_code, count(*) 
    from pgaredm.v_cnf_customer_hst 
    where record_date=trunc(sysdate-1) 
    group by customer_tp_code
"""
curr = conn.cursor()
curr.execute(sql)

# çekilen verileri yazdırma
for row in curr:
    print(row)
    
curr.close()
conn.close()

### pandas ile okuma

In [ ]:
import pandas as pd

conn = connect_db()
df = pd.read_sql(sql, conn)
df

## veri yazma : to_sql

In [ ]:
from bvia_packages.bvia_dwh_processes import dwh_processes

def to_sql(df, table_name, if_exists='append', debug=True):
    conn = connect_db()
    curr = conn.cursor()

    def upload_df(df,table_name):
        with open('/data/home/alperayd/oracle.txt','r') as f:
            auth = f.read().split(';')
        dwh_processes(database = 'SANDBOX', 
                      orcl_user = auth[0], 
                      orcl_pass = auth[1]
                     ) \
                    .write_to_sandbox(data = df, table_name = table_name)
        
    sql=f"select * from ALL_TABLES where TABLE_NAME='{table_name}'"
    df_tbl = pd.read_sql(sql, conn)
    
    if df_tbl.shape[0]==0: 
        upload_df(df, table_name)
        
    elif df_tbl.shape[0]!=0 and if_exists=='replace':
        try:
            curr.execute(f'drop table {table_name}')
        except:
            pass
        upload_df(df, table_name)
        
    elif df_tbl.shape[0]!=0 and if_exists=='append':
        stg_ext = f'_STG{int(np.random.rand()*1000):03}'
        table_name_x = table_name+stg_ext
        
        if debug: print(table_name_x)
        upload_df(df, table_name_x)
        
        sql = f"INSERT INTO {table_name} select * from {table_name_x}"
        curr.execute(sql)
        conn.commit()
        
        curr.execute(f"DROP TABLE {table_name_x}")
        conn.commit()
        
        if debug: print('completed')


In [ ]:
sql = """
    select customer_tp_code, count(*) 
    from pgaredm.v_cnf_customer_hst 
    where record_date=trunc(sysdate-1) 
    group by customer_tp_code
"""
conn = connect_db()
df = pd.read_sql(sql, conn)
df

In [ ]:
df.iloc[-1,0] = 'Bireysel'
df.iloc[ 0,0] = 'Diğer'
df

In [ ]:
to_sql(df, 'KLV_SEGMENT_TMP', if_exists='replace', debug=True)

In [ ]:
df2 = pd.DataFrame([['X', 12345]], columns=['CUSTOMER_TP_CODE','COUNT___'])
df2

In [ ]:
to_sql(df2, 'KLV_SEGMENT_TMP', if_exists='append')